In [23]:
import torch
import torch.nn as nn
import torch.optim as optim
import time
import os
import torchvision
from torchvision import models
import timm
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import time
from tqdm import tqdm
import yaml
import pretrainedmodels
import ssl
from transformers import ViTForImageClassification
import wandb

In [ ]:
with open('/home/disi/ml/intromlproject/config.yaml', 'r') as file:
    config = yaml.safe_load(file)

config = config['config']
config['data_dir'] = config['data_dir'].format(root=config['root'], img_folder=config['img_folder'])
config['save_dir'] = config['save_dir'].format(root=config['root'], model_name=config['model_name'], img_folder=config['img_folder'])
if config['checkpoint'] is not None:
    config['checkpoint'] = config['checkpoint'].format(root=config['root'])
config['device'] = "cuda" if torch.cuda.is_available() else "cpu"
config['project_name'] = config['project_name'].format(model_name=config['model_name'])
config['dataset_name'] = config['dataset_name'].format(img_folder=config['img_folder'])

In [24]:
class SEBlock(nn.Module):
    def __init__(self, in_channels, reduction=16):
        super(SEBlock, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(in_channels, in_channels // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(in_channels // reduction, in_channels, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1, 1)
        return x * y.expand_as(x)

class SEBottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1, downsample=None, reduction=16, dropout_prob=0.5):
        super(SEBottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, planes * self.expansion, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.se = SEBlock(planes * self.expansion, reduction)
        self.downsample = downsample
        self.stride = stride
        self.dropout = nn.Dropout(dropout_prob)

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        out = self.se(out)
        out = self.dropout(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out

class SEResNet50(nn.Module):
    def __init__(self, num_classes=1000, dropout_prob=0.5):
        super(SEResNet50, self).__init__()
        self.num_classes = num_classes
        
        self.inplanes = 64
        resnet = models.resnet50(weights='IMAGENET1K_V2')
        self.conv1 = resnet.conv1
        self.bn1 = resnet.bn1
        self.relu = resnet.relu
        self.maxpool = resnet.maxpool

        self.layer1 = self._make_layer(SEBottleneck, 64, 3)
        self.layer2 = self._make_layer(SEBottleneck, 128, 4, stride=2)
        self.layer3 = self._make_layer(SEBottleneck, 256, 6, stride=2)
        self.layer4 = self._make_layer(SEBottleneck, 512, 3, stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc1 = nn.Linear(512 * SEBottleneck.expansion, 512)
        self.fc2 = nn.Linear(512, num_classes)

        self.dropout = nn.Dropout(p=dropout_prob)

        # Freeze initial layers
        for param in self.conv1.parameters():
            param.requires_grad = False
        for param in self.bn1.parameters():
            param.requires_grad = False
        for param in self.layer1.parameters():
            param.requires_grad = False

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.dropout(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

    def _make_layer(self, block, planes, blocks, stride=1, reduction=16):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, reduction))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, reduction=reduction))

        return nn.Sequential(*layers)

In [25]:
class ViT(nn.Module):
    def __init__(self, num_classes=1000):
        super(ViT, self).__init__()
        self.model_name = 'vit_base_patch16_224'
        self.num_classes = num_classes
        self.model = timm.create_model(self.model_name, pretrained=True, num_classes=num_classes)

        # Define the last layer for classification
        self.model.head = nn.Linear(self.model.head.in_features, self.num_classes)

        self.layer_names = [name for name, _ in self.model.named_parameters()]
        self.total_layers = len(self.layer_names)

        self.layer_names_MSHA = [name for name, _ in self.model.named_parameters() if any(n in name for n in ['blocks', 'attn'])]
        self.total_layers_MSHA = len(self.layer_names_MSHA)
        
        self._initial_freeze_layers()

        self.current_epoch = 0

    def _initial_freeze_layers(self):
        num_to_freeze = int(0.8 * self.total_layers_MSHA)
        for name, param in self.model.named_parameters():
            if any(n in name for n in ['blocks', 'attn']):
                param.requires_grad = True
        for name in self.layer_names_MSHA[:num_to_freeze]:
            param = dict(self.model.named_parameters())[name]
            param.requires_grad = False

    def unfreeze_layers(self, num_unfreeze):
        """Unfreeze `num_unfreeze` layers every epoch."""
        num_to_unfreeze = int(0.1 * self.total_layers_MSHA)
        start_idx = self.current_epoch * num_to_unfreeze
        end_idx = start_idx + num_to_unfreeze
        if end_idx > self.total_layers:
            end_idx = self.total_layers

        for name in self.layer_names_MSHA[start_idx:end_idx]:
            param = dict(self.model.named_parameters())[name]
            param.requires_grad = True

    def print_frozen_layers(self):
        for name, param in self.model.named_parameters():
            print(f"{name}: {'Frozen' if not param.requires_grad else 'Trainable'}")

    def _freeze_non_mhsa_layers(self):
        for name, param in self.model.named_parameters():
            if not any(n in name for n in ['blocks', 'attn']):
                param.requires_grad = False

    def forward(self, x):
        output = self.model(x)
        return output

In [26]:
class ViTFineTuner(nn.Module):
    def __init__(self, num_classes, freeze_layers=True, num_frozen_blocks=6):
        super(ViTFineTuner, self).__init__()
        self.model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224-in21k')
        self.model.classifier = nn.Linear(self.model.classifier.in_features, num_classes)
        
        # Initialize the new classifier layer
        nn.init.xavier_uniform_(self.model.classifier.weight)
        if self.model.classifier.bias is not None:
            nn.init.zeros_(self.model.classifier.bias)
        
        if freeze_layers:
            # Freeze the embedding layer and the first few transformer blocks
            for param in self.model.vit.embeddings.parameters():
                param.requires_grad = False
            for param in self.model.vit.encoder.layer[:num_frozen_blocks].parameters():
                param.requires_grad = False

    def forward(self, x):
        return self.model(x).logits

In [27]:
def get_data_loaders(data_dir, batch_size=32,
                     resize=(256, 256), crop=(224, 224),
                     mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]):
    train_dir = os.path.join(data_dir, 'train')
    val_dir = os.path.join(data_dir, 'val')
    test_dir = os.path.join(data_dir, 'test')

    train_transform = transforms.Compose([
        transforms.Resize(resize),
        transforms.RandomCrop(crop),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),  
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),  
        transforms.ToTensor(),
        transforms.Normalize(mean=mean, std=std)
    ])

    val_transform = transforms.Compose([
        transforms.Resize(resize),
        transforms.CenterCrop(crop),
        transforms.ToTensor(),
        transforms.Normalize(mean=mean, std=std)
    ])

    test_transform = transforms.Compose([
        transforms.Resize(resize),
        transforms.CenterCrop(crop),
        transforms.ToTensor(),
        transforms.Normalize(mean=mean, std=std)
    ])

    train_dataset = datasets.ImageFolder(train_dir, transform=train_transform)
    val_dataset = datasets.ImageFolder(val_dir, transform=val_transform)
    test_dataset = datasets.ImageFolder(test_dir, transform=test_transform)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

    return train_loader, val_loader, test_loader

In [28]:
def evaluate_model(model, data_loader, criterion=None, device='cuda'):
    model.eval()
    correct = 0
    total = 0
    running_loss = 0.0
    with torch.no_grad():
        for inputs, labels in tqdm(data_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            if criterion:
                loss = criterion(outputs, labels)
                running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = correct / total
    if criterion:
        avg_loss = running_loss / len(data_loader.dataset)
        return avg_loss, accuracy
    return accuracy

In [29]:
def save_model(model, file_name):
    # Define the directory
    dir_name = "trained_models/"
    
    # Create the directory if it doesn't exist
    if not os.path.exists(dir_name):
        os.makedirs(dir_name)
    
    # Save the model's state dictionary to the specified file
    file_path = os.path.join(dir_name, file_name)
    torch.save(model.state_dict(), file_path)

In [30]:
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler_gamma, scheduler_step_size, 
                num_epochs, device, patience, checkpoint_path, model_name, dataset_name, save_dir):

    start_time = time.time()

    best_val_loss = float('inf')
    best_val_acc = 0.0
    counter = 0
    
    if scheduler_step_size is not None and scheduler_gamma is not None:
        scheduler_bool = True
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=scheduler_step_size, gamma=scheduler_gamma)
    else:
        scheduler_bool = False

    for epoch in range(num_epochs):
        print("\n", '-'*10)
        if scheduler_bool:
            current_lr = scheduler.get_last_lr()[0]
        else:
            current_lr = optimizer.param_groups[0]['lr']
        print(f'Epoch {epoch+1}/{num_epochs}, Current Learning Rate: {current_lr}')     
        e_start = time.time()
        
        
        # Training phase
        model.train()
        running_loss = 0.0
        for inputs, labels in tqdm(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
        
        epoch_loss = running_loss / len(train_loader.sampler)
        train_acc = evaluate_model(model, train_loader, None, device)
        wandb.log({
                "train/loss": epoch_loss,
                "train/accuracy":train_acc,
                "epoch": epoch
                })
        # Validation phase
        val_loss, val_acc = evaluate_model(model, val_loader, criterion, device)
        wandb.log({
                "val/loss":val_loss,
                "val/accuracy":val_acc,
                "epoch": epoch})

        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Val Loss: {val_loss:.4f}')
        print(f'Train Accuracy: {train_acc:.4f}, Val Accuracy: {val_acc:.4f}')
        print('Epoch time: ', time.time() - e_start)
        
        # Check for best validation accuracy and save model
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            print(f'Best model with val accuracy: {best_val_acc:.4f}')
        
        save_path = os.path.join(checkpoint_path, f"{model_name}_{dataset_name}_epoch{epoch}.pth")
        os.makedirs(save_dir, exist_ok=True)
        # Save the model weights
        torch.save(model.state_dict(), save_path)
        
        # Check for best validation loss and early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            counter = 0
        else:
            counter += 1
            if counter >= patience:
                print(f'Validation loss did not improve for {patience} epochs. Early stopping...')
                break
        
        if scheduler_bool:
            scheduler.step()


    
    print('Training complete. Best validation accuracy: {:.4f}'.format(best_val_acc))
    print('Total time: ', time.time() - start_time)

    return model

In [31]:
model_name = 'senet' # 'vit' or 'senet' or 'densenet' or 'efficientnet'
dataset = 'cub' # 'cub' or 'cars' or 'aircrafts'
data_dir = '../datasets/' + dataset
batch_size = 32
num_epochs = 20
dropout_rate = 0.5
learning_rate = 0.001
momentum = 0.9
weight_decay = 0.001
criteria = 'cross_entropy'
optimz = 'adam' # 'adam' or 'sgd'
scheduler_step_size = 5
scheduler_gamma = 0.1
patience = 5
resize = 256
crop_size = 224
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

In [32]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = len(os.listdir(data_dir + '/train'))

In [34]:
if model_name == 'senet':
    # model = SEResNet50(num_classes=num_classes, dropout_prob = dropout_rate).to(device)
    model = torch.load('pretrained_SENet.pth')
    model.last_linear = nn.Linear(model.last_linear.in_features, num_classes)
    model = model.to(device)
elif model_name == 'vit':
    # model = ViT(num_classes=num_classes).to(device)
    # model = timm.create_model('vit_base_patch16_224', pretrained=True)
    # for name, param in model.named_parameters():
    #     if 'blocks' in name and 'attn' in name:
    #         param.requires_grad = True
    #     else:
    #         param.requires_grad = False
    # model.head = nn.Linear(model.head.in_features, num_classes)
    # model = model.to(device)
    model = ViTFineTuner(num_classes).to(device)
elif model_name == 'efficientnet':
    model = torchvision.models.efficientnet_v2_s(weights=torchvision.models.EfficientNet_V2_S_Weights.IMAGENET1K_V1)
    # Freeze the first layer
    first_layer = model.features[0]
    for param in first_layer.parameters():
        param.requires_grad = False
    # Modify the classifier for the desired number of output classes
    in_features = model.classifier[1].in_features  # Access the in_features of the second layer of the classifier
    model.classifier[1] = nn.Linear(in_features, num_classes)
    model = model.to(device)
    # raise ValueError("Model not found")

Downloading: "http://data.lip6.fr/cadene/pretrainedmodels/senet154-c7b49a05.pth" to /home/disi/.cache/torch/hub/checkpoints/senet154-c7b49a05.pth
100%|██████████| 440M/440M [21:35<00:00, 356kB/s] 


In [36]:
if criteria == 'cross_entropy':
    criterion = nn.CrossEntropyLoss()
else:
    raise ValueError("Criterion not found")

In [37]:
if optimz == 'adam':
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
elif optimz == 'adamw':
    optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
elif optimz == 'sgd':
    optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum, weight_decay=weight_decay)
else:
    raise ValueError("Optimizer not found")

In [38]:
train_loader, val_loader, test_loader = get_data_loaders(data_dir, batch_size, resize, crop_size, mean, std)

In [ ]:
wandb.login()  # @edit
wandb.init(project='FINEGRAINING'+model_name+dataset,
name = 'antonello'
#name=f"{config['model_name']}_{config['dataset_name']}_opt: {config['optimizer']}_batch_size: {config['batch_size']}_lr: {config['learning_rate']}",
# config=config
)

In [ ]:
checkpoint_path = '../EnricoModelli'
checkpoint = None

In [ ]:
from utils.models_init import load_checkpoint
if checkpoint is not None:
    load_checkpoint(model, checkpoint_path, device="cuda")
    print("Checkpoint loaded from" + checkpoint_path)
save_dir = '../EnricoModelli'

In [39]:
model = train_model(model, train_loader, val_loader, criterion, optimizer, scheduler_gamma, scheduler_step_size, 
            num_epochs, device, patience, checkpoint_path, model_name, dataset, save_dir)


 ----------
Epoch 1/20, Current Learning Rate: 0.001


  0%|          | 0/258 [00:00<?, ?it/s]/home/disi/ml/.venv/lib/python3.11/site-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
100%|██████████| 56/56 [00:25<00:00,  2.20it/s]


Epoch 1/20, Loss: 5.0809, Val Loss: 598.0891
Train Accuracy: 0.0112, Val Accuracy: 0.0096
Epoch time:  571.1983280181885
Best model saved with val accuracy: 0.0096

 ----------
Epoch 2/20, Current Learning Rate: 0.001


  3%|▎         | 9/258 [00:17<07:55,  1.91s/it]


KeyboardInterrupt: 

In [ ]:
accuracy = evaluate_model(model, test_loader, criterion = None, device = device)